<a href="https://colab.research.google.com/github/bhaveshkolhe/QuadB-Tech-Task/blob/main/QuadB_Tech_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import hashlib
import time
import json

class Block:
    def __init__(self, index, transactions, previous_hash, nonce=0):
        self.index = index
        self.timestamp = time.time()
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.hash = self.compute_hash()

    def compute_hash(self):
        """
        Creates a SHA-256 hash of the block's contents
        """
        block_string = json.dumps({
            "index": self.index,
            "timestamp": self.timestamp,
            "transactions": self.transactions,
            "previous_hash": self.previous_hash,
            "nonce": self.nonce
        }, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    def mine_block(self, difficulty):
        """
        Proof-of-work: Find a nonce that makes the hash start with '0'*difficulty
        """
        while not self.hash.startswith('0' * difficulty):
            self.nonce += 1
            self.hash = self.compute_hash()

class Blockchain:
    def __init__(self, difficulty=4):
        self.chain = []
        self.difficulty = difficulty
        self.create_genesis_block()

    def create_genesis_block(self):
        """
        Creates the first block in the blockchain
        """
        genesis_block = Block(0, ["Genesis Transaction"], "0")
        genesis_block.mine_block(self.difficulty)
        self.chain.append(genesis_block)

    def add_block(self, transactions):
        """
        Adds a new block to the blockchain after mining
        """
        previous_hash = self.chain[-1].hash
        new_block = Block(len(self.chain), transactions, previous_hash)
        new_block.mine_block(self.difficulty)
        self.chain.append(new_block)

    def is_chain_valid(self):
        """
        Validates the integrity of the blockchain
        """
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]

            # Check if current block's previous_hash matches previous block's hash
            if current_block.previous_hash != previous_block.hash:
                return False

            # Recompute current block's hash to verify its validity
            if current_block.hash != current_block.compute_hash():
                return False

        return True

    def print_chain(self):
        """
        Prints all blocks in the blockchain
        """
        for block in self.chain:
            print(f"Block #{block.index} [")
            print(f"  Hash: {block.hash}")
            print(f"  Previous Hash: {block.previous_hash}")
            print(f"  Transactions: {block.transactions}")
            print(f"  Nonce: {block.nonce}")
            print("]")

def demonstrate_tampering(blockchain):
    """
    Demonstrates how tampering with a block invalidates the chain
    """
    if len(blockchain.chain) >= 2:
        print("\nTampering with block 1...")
        blockchain.chain[1].transactions = ["Tampered transaction"]
        print("Blockchain valid after tampering?", blockchain.is_chain_valid())

if __name__ == "__main__":
    # Initialize blockchain with difficulty level 4
    blockchain = Blockchain(difficulty=4)

    # Add some blocks with transactions
    blockchain.add_block(["Alice sent Bob 1 BTC", "Bob sent Charlie 0.5 BTC"])
    blockchain.add_block(["Charlie sent Dave 0.2 BTC"])

    # Print the blockchain
    blockchain.print_chain()

    # Validate the blockchain
    print("\nBlockchain valid?", blockchain.is_chain_valid())

    # Demonstrate tampering
    demonstrate_tampering(blockchain)

Block #0 [
  Hash: 00000d553ad448db63d2673ce0456f713f3780dc803a6836b983ed220260619b
  Previous Hash: 0
  Transactions: ['Genesis Transaction']
  Nonce: 6234
]
Block #1 [
  Hash: 0000ab35753487ba0c9f624d27eca2b1641d83941e2929fd12aa7407452a4fdd
  Previous Hash: 00000d553ad448db63d2673ce0456f713f3780dc803a6836b983ed220260619b
  Transactions: ['Alice sent Bob 1 BTC', 'Bob sent Charlie 0.5 BTC']
  Nonce: 8482
]
Block #2 [
  Hash: 000096d962d19333f7d1fb64f02459e049c5cd3b420eea17bf1c9ed347b531b8
  Previous Hash: 0000ab35753487ba0c9f624d27eca2b1641d83941e2929fd12aa7407452a4fdd
  Transactions: ['Charlie sent Dave 0.2 BTC']
  Nonce: 37541
]

Blockchain valid? True

Tampering with block 1...
Blockchain valid after tampering? False
